## Markov Text Generator

Random text generation with a simple Markov language model

### What is a Markov chain?

- A Markov chain is a stochastic process
- Markov chains use an internal state model to judge the probability of any state based on the current state
- Markov chains have the Markov property: The prediction depends only on the current state.

Now, let's build a simple text generator. We want to predict the most probable next word, based on a limited set of current words.

## The implementation

Let's start with a few basics. We'll use `random` to randomize output, and `pathlib.Path` to access text files. The function `sys.getsizeof` is used later to evaluate memory requirements of our language model. Finally, `matplotlib` is used to visualize experimental results, later on. **Note:** You might have to install matplotlib on your system, first, using pip (or similar).

In [1]:
import random
from pathlib import Path
from sys import getsizeof
%matplotlib notebook
import matplotlib.pyplot as plt

### 1. Preparing the text

First, select an appropriate data set. You'll want a file of plain text. Potentially you'll have to preprocess whatever data you choose to operate on.

#### Task

Find a data set online, prepare it as raw ASCII text, and load it in the code block below.

In [2]:
text_file = Path('/content/data.txt')

Next, you'll need a function **_file_to_words_**, which reads the `text_file` above and produces a list of words from the file. This is used as training data, to build the language model.

#### Task

Complete the function below appropriately.

In [7]:
def file_to_words(file: Path):
    with file.open(mode='rt', encoding='utf-8') as f:
        return f.read().split()


try:
    words = file_to_words(text_file)
except Exception as e:
    print("Something is wrong with your file")
    print(e)
else:
    print("First 100 words:")
    print(words[:100])

First 100 words:
['Main', 'menu', 'WikipediaThe', 'Free', 'Encyclopedia', 'Search', 'Wikipedia', 'Search', 'Create', 'account', 'Log', 'in', 'Personal', 'tools', 'Contents', 'hide', '(Top)', '2016', 'election', 'Transition', 'period,', 'inauguration,', 'and', 'first', '100', 'days', 'Administration', 'Toggle', 'Administration', 'subsection', 'Judicial', 'appointments', 'Toggle', 'Judicial', 'appointments', 'subsection', 'Leadership', 'style', 'Toggle', 'Leadership', 'style', 'subsection', 'Domestic', 'affairs', 'Toggle', 'Domestic', 'affairs', 'subsection', 'Foreign', 'affairs', 'Toggle', 'Foreign', 'affairs', 'subsection', 'Russia', 'and', 'related', 'investigations', 'Toggle', 'Russia', 'and', 'related', 'investigations', 'subsection', 'Ethics', 'Toggle', 'Ethics', 'subsection', 'Elections', 'during', 'the', 'Trump', 'presidency', 'Toggle', 'Elections', 'during', 'the', 'Trump', 'presidency', 'subsection', 'Historical', 'evaluations', 'and', 'public', 'opinion', 'See', 'also', 'Refer

### 2. Tuples: Groups of words

While we have the text subdivided into words now, we want to experiment with various context windows. For that, we need to group words into tuples. Since we want to experiment with the context window, the next function should be generic, taking the size of the context window as a parameter.

#### Example

The sentence "What a lovely day in February" (6 words) split with a context window of three yields four tuples:

- (What, a, lovely)
- (a, lovely, day)
- (lovely, day, in)
- (day, in, February)

**Note:** The last word is later on going to be the predicted word. So, if we split with a window size of three, here, we will later on have a context window of only two words (the third word being the predicted one).

#### Task

Complete the function below.

In [23]:
def tuples(words, window_size):
    """Generates tuples from the given data string.
    So if our string was "What a lovely day", we'd generate
    (What, a, lovely) and then (a, lovely, day).
    """
    tuples_list = []
    for i in range(len(words) - window_size + 1):
        tuple_item = tuple(words[i:i+window_size])
        tuples_list.append(tuple_item)
    return tuples_list

test_sentence = "What a lovely day in February".split()
data = list(tuples(test_sentence, 3))[0]
assert(data == ('What', 'a', 'lovely'))
data = list(tuples(test_sentence, 4))[1]
assert(data == ('a', 'lovely', 'day', 'in'))


('What', 'a', 'lovely')
('a', 'lovely', 'day', 'in')


### 3. A primitive database

We will create a primitive database to create the Markov chain from the tuples we just got. We'll use a Python dictionary for that. A dictionary can be used as an adjacency list, where the key is the current node and the value is a list of nodes adjacent to the current node.

#### Example

The sentence "I like coffee, I like tea" with a context of one word would produce the following dictionary:
```
{
  ('I'): [('like')],
  ('like'): [('coffee,'), ('tea')],
  ('coffee,'): [('I')],
  ('tea'): [],
}
```

**Note:** You will have to capture the probabilities of the next node. There's multiple ways of implementing this. Consider, how you will do it, then proceed with your implementation.

#### Task

Complete the following function.

In [35]:
def generate_database(data):
  result = {}
  for t in data:
    l = list (t)
    key = tuple(l[:-1])
    word = l[-1]
    if key in result:
      result [key].append(word)
    else:
      result[key] = [word]
  return result


test_sentence = "I like coffee, I like tea".split()
data = list(tuples(test_sentence, 2))
database = generate_database(data)
print (database)
assert(('tea') in database[('like' ,)])


{('I',): ['like', 'like'], ('like',): ['coffee,', 'tea'], ('coffee,',): ['I']}


### 4. Markov text generation

Now, let's generate some text. Based on a prompt, we will predict next tokens repeatedly, until either a certain amount of text is produced, or the chain reaches an end (i.e., a token with no follow-up tokens).

#### Task

Complete the following function

In [48]:
def generate_markov_text(prompt: str, database: dict, text_size: int = 50):
  front = prompt.split()[-3:]
  while text_size > 0:
    t=tuple(front)
    word=random.choice(database[t])
    front.pop(0)
    front.append(word)
    text_size-=1
    yield word

words = file_to_words(text_file)
data=tuples(list(words), 4)
database = generate_database(data)
prompt = "and scholars and historians"

for word in generate_markov_text(prompt, database):
    print(word, end=' ')

rank his presidency as one of the worst in American history. 2016 election Main articles: Donald Trump on social media and Donald Trump sexual misconduct allegations The Trump administration ignored detailed plans on how to mass-produce protective respirator masks under a program that had been launched by the Obama administration 

### 5. Experimentation

Now, let's investigate further.

#### Task

Use the `getsizeof` function to compare memory requirements for various context windows. Experiment with context windows between 1+1 and 20+1 (the "+1" being the predicted next word, respectively). Use `matplotlib` to draw a bar chart of memory requirements based on the training text data you are using.